# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.
This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

This notebook illustrates how to:
- create a project
- extract training frames
- label the frames
- plot the labeled images
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video 
- plot the trajectories

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## Note: this is for use with 3d slices of data in Zimmer lab format 

For now I'm testing non-GUI created annotations. Thus, I've deleted the label-creation portions of this demo and replaced them with importing

## Create a new project

It is always good idea to keep the projects seperate if you want to use different networks to analze your data. You should use one project if you are tracking similar subjects/items even if in different environments. This function creates a new project with sub-directories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos (for lableing more data) to the project at any stage of the project. 

In [9]:
import deeplabcut

In [2]:
task='WormTest' # Enter the name of your experiment Task
experimenter='Charlie' # Enter the name of the experimenter
video=['wb_immobilized/KU20170626-TKU721_nostim_ctrl_w2_part1.ome.tiff'] # Enter the paths of your videos OR FOLDER you want to grab frames from.

#path_config_file=deeplabcut.create_new_project(task,experimenter,video,copy_videos=True) 

# NOTE: The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

In [3]:
print(path_config_file)

C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\config.yaml


## Now, go edit the config.yaml file that was created! 
Add your body part labels, edit the number of frames to extract per video, etc. 

#### Note that you can see more information about ANY function by adding a ? at the end,  i.e. 

## Save individual volumes from ome.tiff movie

In [119]:
video_fname = '{}\\videos\\KU20170626-TKU721_nostim_ctrl_w2_part1.ome.tiff'.format(home)

with tifffile.TiffFile(video_fname) as vid:
    #print(vid)
    #tmp = vid.asarray(series=0,key=[1,13])
    #print(vid.ome_metadata)
#    print(vid.ome_metadata.keys())
#    print(vid.ome_metadata['Image']['Pixels'].keys())
    print(vid.ome_metadata['Image']['Pixels']['SizeT'])
    print(vid.ome_metadata['Image']['Pixels']['SizeY'])
    print(vid.ome_metadata['Image']['Pixels']['SizeZ'])
    
#print(tmp.shape)

854
512
13


In [205]:
#
# Basically this is a custom extract_frames() function
#

from pathlib import Path
from deeplabcut.utils import auxiliaryfunctions
import tifffile

def movie2volume(path_config_file, which_vol=None):
    """
    Takes a video filename, which is a large ome-tiff file, and saves a volume in the folder given by 'out_folder'
    """
    
    config_file = Path(path_config_file).resolve()
    cfg = auxiliaryfunctions.read_config(config_file)
    print("Config file read successfully.")
    
    video_fname = [i for i in cfg['video_sets'].keys()][0] # Assume one video for now (will be giant, ~3GB)
    print(video_fname)
    
    # Read basic metadata
    with tifffile.TiffFile(video_fname) as vid:
        mdat = vid.ome_metadata['Image']['Pixels']
        nz, nt = mdat['SizeZ'], mdat['SizeT']
    
    # Get volume indices to save
    if which_vol is None:
        which_vol = [0]
    
    for i_vol in which_vol:
        print("Read volume {}/{}".format(i_vol, len(which_vol)))
        
        # Convert scalar volume label to the sequential frames
        # Note: this may change for future input videos!
        vol_indices = list(range(i_vol*nz, i_vol*nz + nz))

        # Read and make output name
        this_volume = tifffile.imread(video_fname, key=vol_indices)
        output_name = 'img{}.tif'.format(i_vol)

        # Save in output folder
        fname = Path(video_fname)
        output_path = Path(path_config_file).parents[0] / 'labeled-data' / fname.stem

        tifffile.imsave(str(output_path) + '\\' + output_name, this_volume)

        print('Saved volume to {}\\{}'.format(output_path, output_name))
    

In [206]:
movie2volume(path_config_file, list(range(1,5)))

Config file read successfully.
C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\videos\KU20170626-TKU721_nostim_ctrl_w2_part1.ome.tiff
Read volume 1/4
Saved volume to C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\labeled-data\KU20170626-TKU721_nostim_ctrl_w2_part1.ome\img1.tif
Read volume 2/4
Saved volume to C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\labeled-data\KU20170626-TKU721_nostim_ctrl_w2_part1.ome\img2.tif
Read volume 3/4
Saved volume to C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\labeled-data\KU20170626-TKU721_nostim_ctrl_w2_part1.ome\img3.tif
Read volume 4/4
Saved volume to C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\labeled-data\KU20170626-TKU721_nostim_ctrl_w2_part1.ome\img4.

# Import annotations from MATLAB

In [5]:
import h5py
import os
import pandas as pd
import glob
import numpy as np
from pathlib import Path
from deeplabcut.utils import auxiliaryfunctions


In [6]:
# For now, just have the labels in the example folder
home = os.path.dirname(path_config_file)
label_fname = '{}\\wbstruct.mat'.format(home)
with h5py.File(label_fname, 'r') as mat:
    #print(mat.keys())
    #print(mat['mask'].shape)
    #print(mat['simple'].keys())
    print(mat['simple']['x'].shape)
    print(mat['simple']['x'][0,:])
    print(mat['simple']['x'][0,:].shape)
    #print(mat['simple']['y'][:])
    #print(mat['simple']['z'].shape)
    print(max(mat['simple']['z']))
    print(mat['simple']['nn'][0][0])


(110, 18)
[75. 75. 75. 76. 76. 76. 76. 76. 76. 76. 76. 76. 76. 76. 76. 76. 76. 76.]
(18,)
[12.]
110.0


In [13]:
def wb_tracker2dlc_format(path_config_file):
    """
    Converts Zimmer Whole-Brain tracker data for immobilized worms into training data for DeepLabCut
    """
    
    # Build filenames
    home = os.path.dirname(path_config_file)
    wb_fname = '{}\\wbstruct.mat'.format(home)
    
    config_file = Path(path_config_file).resolve()
    cfg = auxiliaryfunctions.read_config(config_file)
    print("Config file read successfully.")
    
    # Note: the labeled-data subfolder has the entire video name as the folder name
    video_fname = [i for i in cfg['video_sets'].keys()][0] # Assume one video for now
    fname = Path(video_fname)
    output_path = str(Path(path_config_file).parents[0] / 'labeled-data' / fname.stem)
    output_path = output_path + '\\'
    print('Looking in folder {}'.format(output_path))
    
    # Get list of images
    imlist=[]
    imtype = '*.tif'
    imlist.extend([fn for fn in glob.glob(os.path.join(output_path,imtype)) if ('labeled.png' not in fn)])

    if len(imlist)==0:
        print("No images found!!")

    index = np.sort(imlist)
    print('Working on folder: {}'.format(os.path.split(str(output_path))[-1]))
    relativeimagenames=['labeled'+n.split('labeled')[1] for n in index]
    
    # Build dataset using pandas; copied from from labeling_toolbox.py
    scorer = cfg['scorer']
    with h5py.File(wb_fname, 'r') as mat:
        num_neurons = int(mat['simple']['nn'][0][0])
        x = mat['simple']['x'][:,0] # NOTE: a flip is required here; not sure why
        y = mat['simple']['y'][:,0]
        z = mat['simple']['z'][:]
        coords = np.empty((len(index),3,))
        dataFrame = None
        
        for i in range(num_neurons):
            bodypart = 'neuron{}'.format(i)
             
            # Note: this requires a flip in the y direction; not sure why
            x_sz = 272
            coords[:] = np.array([x_sz-x[i], y[i], z[i][0]])

            index = pd.MultiIndex.from_product([[scorer], [bodypart], ['x', 'y', 'z']],names=['scorer', 'bodyparts', 'coords'])
            #print(index)
            print(coords)
            frame = pd.DataFrame(coords, columns = index, index = relativeimagenames)
            dataFrame = pd.concat([dataFrame, frame],axis=1)
        
    dataFrame.to_csv(os.path.join(output_path,"CollectedData_" + scorer + ".csv"))
    dataFrame.to_hdf(os.path.join(output_path,"CollectedData_" + scorer + '.h5'),'df_with_missing',format='table', mode='w')

In [14]:
wb_tracker2dlc_format(path_config_file)

Config file read successfully.
Looking in folder C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\labeled-data\KU20170626-TKU721_nostim_ctrl_w2_part1.ome\
Working on folder: 
[[197.  25.   4.]
 [197.  25.   4.]
 [197.  25.   4.]
 [197.  25.   4.]
 [197.  25.   4.]]
[[163.  28.   8.]
 [163.  28.   8.]
 [163.  28.   8.]
 [163.  28.   8.]
 [163.  28.   8.]]
[[197.  45.  10.]
 [197.  45.  10.]
 [197.  45.  10.]
 [197.  45.  10.]
 [197.  45.  10.]]
[[194.  46.   5.]
 [194.  46.   5.]
 [194.  46.   5.]
 [194.  46.   5.]
 [194.  46.   5.]]
[[171.  46.   8.]
 [171.  46.   8.]
 [171.  46.   8.]
 [171.  46.   8.]
 [171.  46.   8.]]
[[185.  59.   4.]
 [185.  59.   4.]
 [185.  59.   4.]
 [185.  59.   4.]
 [185.  59.   4.]]
[[167.  63.   4.]
 [167.  63.   4.]
 [167.  63.   4.]
 [167.  63.   4.]
 [167.  63.   4.]]
[[206.  63.   5.]
 [206.  63.   5.]
 [206.  63.   5.]
 [206.  63.   5.]
 [206.  63.   5.]]
[[190.  66.   4.]
 [190.  66.   4.]
 [19

[[268. 425.   7.]
 [268. 425.   7.]
 [268. 425.   7.]
 [268. 425.   7.]
 [268. 425.   7.]]


# Automatically update the config file

In [248]:
import csv

def wb_tracker2config_names(path_config_file):
    """
    Automatically updates the config file with the proper number of neurons, and deletes any other default bodyparts.
    Only affects the "bodyparts" field
    """
    
    # Get number of neurons from wbstruct
    home = os.path.dirname(path_config_file)
    wb_fname = '{}\\wbstruct.mat'.format(home)
    with h5py.File(wb_fname, 'r') as mat:
        num_neurons = int(mat['simple']['nn'][0][0])
    
    # Read in entire config file into a list
    config_rows = []
    with open(path_config_file) as config:
        c_reader = csv.reader(config)#, delimiter=' ')
        for row in c_reader:
            config_rows.append(row)
    
    ## Delete the current bodypart lines
    delete_these_rows = False
    config_rows_edit = config_rows.copy()
    for row in config_rows:
        if row == ['bodyparts:']:
            delete_these_rows = True # Start deleting next row
        elif row == ['start: 0']:
            delete_these_rows = False # Do not delete this row, or others
            break
        elif delete_these_rows == True:
            # Don't delete either of the two above, but only in between those rows
            config_rows_edit.remove(row)
    
    ## Add in the named neuron lines
    # Using "list slicing" https://www.geeksforgeeks.org/python-insert-list-in-another-list/
    new_names = [['- neuron{}'.format(i)] for i in range(num_neurons)]
    insert_index = config_rows_edit.index(['start: 0'])
    config_rows_edit[insert_index:insert_index] = new_names
    
    ## Write the file again
    if True:
        with open(path_config_file, 'w', newline='') as config:
            c_writer = csv.writer(config)
            for row in config_rows_edit:
                c_writer.writerow(row)
    #for row in config_rows_edit:
        #c_writer.writerow(row)
     #   print(row[:])
    
    

In [249]:
wb_tracker2config_names(path_config_file)

## Check the labels

[OPTIONAL] Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [15]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Charlie.
C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\labeled-data\KU20170626-TKU721_nostim_ctrl_w2_part1.ome_labeled  already exists!
(13, 512, 272)
They are stored in the following folder: C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\labeled-data\KU20170626-TKU721_nostim_ctrl_w2_part1.ome_labeled.
(13, 512, 272)
(13, 512, 272)
(13, 512, 272)
(13, 512, 272)
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use relauch the labeling GUI to move them around, save, and re-plot!

## TODO: Create a training dataset

This function generates the training data information for network training based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles if they want to benchmark the performance (typcailly, 1 is what you will set, so you pass nothing!). 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**. For most all use cases we have seen, the defaults are perfectly fine.

Now it is the time to start training the network!

In [1]:
# IF THE KERNEL IS RESET
import deeplabcut
path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

In [16]:
deeplabcut.create_training_dataset(path_config_file)
#remember, there are several networks you can pick, the default is resnet-50!

C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\training-datasets\iteration-0\UnaugmentedDataSet_WormTestMay21  already exists!
Recognized zslice training data; using custom function
C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\dlc-models\iteration-0\WormTestMay21-trainset95shuffle1  already exists!
C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\dlc-models\iteration-0\WormTestMay21-trainset95shuffle1/train  already exists!
C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\dlc-models\iteration-0\WormTestMay21-trainset95shuffle1/test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95, 1, (array([1, 3, 0, 2]), array([4])))]

In [202]:
deeplabcut.create_training_dataset?

## Start training:

This function trains the network for a specific shuffle of the training dataset. 

In [17]:
deeplabcut.train_network(path_config_file, displayiters=5, saveiters=100)

Config:
{'Task': None,
 'TrainingFraction': None,
 'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26],
                [27],
                [28],
                [29],
                [30],
                [31],
                [32],
                [33],
                [34],
                [35],
                [36],
                [37],
                [38],
                [39],
                [40],
                [41],
                [42],
              

Switching batchsize to 1, as default/tensorpack/deterministic loaders do not support batches >1. Use imgaug loader.
Starting with standard pose-dataset loader.
Initializing ResNet
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Loading ImageNet-pretrained resnet_50
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from c:\users\charl\documents\current_work\deeplabcut_etc\deeplabcut-fork\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt
Display_iters overwritten as 5
Save_iters overwritten as 100
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Cha

iteration: 5 loss: 0.8755 lr: 0.005
iteration: 10 loss: 0.6284 lr: 0.005
iteration: 15 loss: 0.3695 lr: 0.005
iteration: 20 loss: 0.1382 lr: 0.005
iteration: 25 loss: 0.1062 lr: 0.005
iteration: 30 loss: 0.0748 lr: 0.005
iteration: 35 loss: 0.0625 lr: 0.005
iteration: 40 loss: 0.0577 lr: 0.005
iteration: 45 loss: 0.0548 lr: 0.005
iteration: 50 loss: 0.0615 lr: 0.005
iteration: 55 loss: 0.0638 lr: 0.005
iteration: 60 loss: 0.0594 lr: 0.005
iteration: 65 loss: 0.0464 lr: 0.005
iteration: 70 loss: 0.0503 lr: 0.005
iteration: 75 loss: 0.0491 lr: 0.005
iteration: 80 loss: 0.0562 lr: 0.005
iteration: 85 loss: 0.0493 lr: 0.005
iteration: 90 loss: 0.0462 lr: 0.005
iteration: 95 loss: 0.0571 lr: 0.005
iteration: 100 loss: 0.0490 lr: 0.005
iteration: 105 loss: 0.0513 lr: 0.005
iteration: 110 loss: 0.0508 lr: 0.005
iteration: 115 loss: 0.0496 lr: 0.005
iteration: 120 loss: 0.0540 lr: 0.005
iteration: 125 loss: 0.0432 lr: 0.005
iteration: 130 loss: 0.0476 lr: 0.005
iteration: 135 loss: 0.0396 lr: 

KeyboardInterrupt: 

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [1]:
# IF THE KERNEL IS RESET
import deeplabcut
path_config_file = 'C:\\Users\\charl\\Documents\\Current_work\\DeepLabCut_etc\\DeepLabCut-fork\\examples\\WormTest-Charlie-2020-05-21\\config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

In [18]:
deeplabcut.evaluate_network(path_config_file, plotting=True)

Config:
{'Task': None,
 'TrainingFraction': None,
 'all_joints': [[0],
                [1],
                [2],
                [3],
                [4],
                [5],
                [6],
                [7],
                [8],
                [9],
                [10],
                [11],
                [12],
                [13],
                [14],
                [15],
                [16],
                [17],
                [18],
                [19],
                [20],
                [21],
                [22],
                [23],
                [24],
                [25],
                [26],
                [27],
                [28],
                [29],
                [30],
                [31],
                [32],
                [33],
                [34],
                [35],
                [36],
                [37],
                [38],
                [39],
                [40],
                [41],
                [42],
              

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21/evaluation-results/  already exists!
C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\evaluation-results\iteration-0\WormTestMay21-trainset95shuffle1  already exists!
Running  DLC_resnet50_WormTestMay21shuffle1_200  with # of trainingiterations: 200
This net has already been evaluated!
Plotting...(attention scale might be inconsistent in comparison to when data was analyzed; i.e. if you used rescale)
C:\Users\charl\Documents\Current_work\DeepLabCut_etc\DeepLabCut-fork\examples\WormTest-Charlie-2020-05-21\evaluation-results\iteration-0\WormTestMay21-trainset95shuffle1\LabeledImages_DLC_resnet50_WormTestMay21shuffle1_200_snapshot-200  already exists!


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.96s/it]


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [0]:
videofile_path = ['videos/video3.avi','videos/video4.avi'] #Enter a folder OR a list of videos to analyze.

deeplabcut.analyze_videos(path_config_file,videofile_path, videotype='.avi')

## Extract outlier frames [optional step]

This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. This step has many options, so please look at:

In [3]:
deeplabcut.extract_outlier_frames?

In [0]:
deeplabcut.extract_outlier_frames(path_config_file,['/videos/video3.avi']) #pass a specific video

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset. 

In [0]:
%gui wx
deeplabcut.refine_labels(path_config_file)

**NOTE:** Afterwards, if you want to look at the adjusted frames, you can load them in the main GUI by running: ``deeplabcut.label_frames(path_config_file)``

(you can add a new "cell" below to add this code!)

#### Once all folders are relabeled, check the labels again! If you are not happy, adjust them in the main GUI:

``deeplabcut.label_frames(path_config_file)``

Check Labels:

``deeplabcut.check_labels(path_config_file)``

In [0]:
#NOW, merge this with your original data:

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refinement of labels and appending them to the original dataset, this creates a new iteration of training dataset. This is automatically set in the config.yaml file, so let's get training!

In [0]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

THIS HAS MANY FUN OPTIONS! 

``deeplabcut.create_labeled_video(config, videos, videotype='avi', shuffle=1, trainingsetindex=0, filtered=False, save_frames=False, Frames2plot=None, delete=False, displayedbodyparts='all', codec='mp4v', outputframerate=None, destfolder=None, draw_skeleton=False, trailpoints=0, displaycropped=False)``

So please check:

In [4]:
deeplabcut.create_labeled_video?

In [0]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [0]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)